# Interactive Conditional Samples (GPT-2)

## Install Required Libraries

In [1]:
#Install Required Libraries
!pip3 install -q -r requirements.txt

In [2]:
#Download the models
#Model Options: 117M or 345M
!python3 download_model.py 117M

Fetching checkpoint: 1.00kit [00:00, 961kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 38.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 281kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:10, 49.2Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 1.47Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 11.5Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 33.6Mit/s]                                                       


## Import Libraries

In [3]:
import json
import os
import numpy as np
import tensorflow as tf

In [4]:
import model, sample, encoder

## Function for Interactive Model

In [10]:
def interact_model(
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',    
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
     :models_dir : path to parent folder containing model subfolders
     (i.e. contains the <model_name> folder)     
    """
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)


## Run Model

In [ ]:
interact_model(
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    models_dir='models',    
)

INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
Model prompt >>> This is a sample model
======================================== SAMPLE 1 ========================================
 in which most instruments, turbo cars and transportation equipment are hard to place in a safe location. Because every part has a little bit of risk, it is always best to opt for a compact model for full context — for the overload risk less, and the less visible it is, the better.

However, in the real world, where in the past many people with big potential were safest stockpiled in selecting a high master control unit, these days the landing jumbo field horn and other equipment is reserved for highly established airframes, even to the tip of the nose, and it has become more popular for small, bright 50s hosiery trucks.

One way to find out what makes people afraid is to visit racebook, where you can find real studies on how people feel when things are going well and by whom directly, but it 